### Generate Demo Data

The point of this notebook is to extract and slice down A58 data into an approximately 1 minute segment. We'll take the Frames 14000-16000

In [60]:
from caltrig.core.backend import open_minian
import os

input_path_data = './data/05.2023_Tenth_group-AA058_D1-2023_05_05-11_02_42-Miniscope_2-S4/minian'
input_path_video = './data/05.2023_Tenth_group-AA058_D1-2023_05_05-11_02_42-Miniscope_2-S4/minian_intermediate'
input_path_timestamp = './data/05.2023_Tenth_group-AA058_D1-2023_05_05-11_02_42-Miniscope_2-S4/AA058_D1_S4_behavior_ms.csv'
behavior_video_timestamps = './data/05.2023_Tenth_group-AA058_D1-2023_05_05-11_02_42-Miniscope_2-S4/behavior_video/timeStamps.csv'

output_path_data = './data/Demo Data/data'
output_path_video = './data/Demo Data/video'
output_path_timestamp = './data/Demo Data'

In [61]:
data_types = ["C", "S", "DFF", "YrA", "f", "E"]

data = open_minian(input_path_data)

In [ ]:

for type in data_types:
    sliced = data[type].sel(frame=slice(14000, 18000))
    # Rewrite the frames coordinates to align with the sliced data
    sliced = sliced.assign_coords(frame=("frame", range(sliced.frame.size)))
    path = os.path.join(output_path_data, sliced.name + ".zarr")
    sliced.to_zarr(path, compute=True, mode="w", consolidated=False)

In [ ]:
video_data_types = ["varr", "Y_fm_chk", "Y_hw_chk"]
video_data = open_minian(input_path_video)

for type in video_data_types:
    sliced = video_data[type].sel(frame=slice(14000, 18000))
    # Rewrite the frames coordinates to align with the sliced data
    sliced = sliced.assign_coords(frame=("frame", range(sliced.frame.size)))
    path = os.path.join(output_path_video, sliced.name + ".zarr")
    sliced.to_zarr(path, compute=True, mode="w", consolidated=False)

In [64]:
length = data["C"].shape[1]

Because the behavior video is shot at a different fps than the calcium recording we need to approximate the sliced behavior video

In [ ]:
# Finally we need to open the csv timestamp file and use frames 14000 to 16000 and save it to the output path
import pandas as pd
timestamps = pd.read_csv(input_path_timestamp)
timestamps = timestamps.iloc[14000:18001]
timestamps.to_csv(os.path.join(output_path_timestamp, "timeStamps.csv"), index=False)


In [66]:
timestamps["Time Stamp (ms)"].iloc[-1]

3223092

In [67]:
behavior_length = video_data["behavior_video"].shape[0]
behavior_length
timestamp_start = timestamps["Time Stamp (ms)"].iloc[0]
timestamp_end = timestamps["Time Stamp (ms)"].iloc[-1]

behavior_timestamps = pd.read_csv(behavior_video_timestamps)
# We need to find the closest index corresponding to the start and end timestamps
frame_start = (behavior_timestamps["Time Stamp (ms)"] - timestamp_start).abs().idxmin()
frame_end = (behavior_timestamps["Time Stamp (ms)"] - timestamp_end).abs().idxmin()

In [68]:
behavior = video_data["behavior_video"].sel(frame=slice(frame_start, frame_end)).chunk({"frame": 100, "height": -1, "width": -1})
# Rewrite the frames coordinates to align with the sliced data
behavior = behavior.assign_coords(frame=("frame", range(behavior.frame.size)))
path = os.path.join(output_path_video, "behavior_video.zarr")
behavior.to_zarr(path, compute=True, mode="w", consolidated=False)

In [69]:
print(behavior.shape)
video_data = open_minian(output_path_video)
bv = video_data["behavior_video"]
bv

(1007, 480, 640)


<xarray.DataArray 'behavior_video' (frame: 1007, height: 480, width: 640)>
dask.array<from-zarr, shape=(1007, 480, 640), dtype=uint8, chunksize=(100, 480, 640), chunktype=numpy.ndarray>
Coordinates:
  * frame    (frame) int32 0 1 2 3 4 5 6 ... 1000 1001 1002 1003 1004 1005 1006
  * height   (height) int32 0 1 2 3 4 5 6 7 ... 472 473 474 475 476 477 478 479
  * width    (width) int32 0 1 2 3 4 5 6 7 8 ... 632 633 634 635 636 637 638 639